# Filtered Backprojection Cone Beam Reconstruction
## With Configurable Filter Sizes and Types

This notebook implements true filtered backprojection for EPID dose reconstruction with:
- Configurable filter types (Shepp-Logan, Ram-Lak, Cosine, Hamming)
- Adjustable filter sizes (0.1 to 1.0 factor)
- Global mask for consistent processing
- Comparison between different reconstruction approaches

In [1]:
import math
import os
from pydicom import dcmread, dcmwrite
import matplotlib.pyplot as plt
import numpy as np
from tqdm import tqdm
from scipy.fft import fft, ifft
from concurrent.futures import ThreadPoolExecutor
import multiprocessing
import time
import warnings
warnings.filterwarnings('ignore')

PI = math.pi
print(f"Available CPU cores: {multiprocessing.cpu_count()}")

# Try to import numba for JIT compilation
try:
    from numba import jit, prange
    NUMBA_AVAILABLE = True
    print("Numba JIT compilation available")
except ImportError:
    NUMBA_AVAILABLE = False
    print("Numba not available - using pure NumPy")
    def jit(func):
        return func
    prange = range

Available CPU cores: 24
Numba JIT compilation available


## Filter Visualization and Comparison

In [ ]:
def filter_SL_broken(N, d):
    """Original Shepp-Logan filter - BROKEN VERSION"""
    fh_SL = np.zeros(N)
    for k1 in range(0, N, 1):
        fh_SL[k1] = -2.0/(PI*PI*d*d*(4*(k1-N/2.0)**2-1))
        
    # This line overwrites everything with zeros! 
    fh_SL = np.zeros(N)  # ❌ THIS IS THE BUG!
    return fh_SL

def filter_SL(N, d, filter_size_factor=1.0):
    """FIXED Shepp-Logan filter with configurable size"""
    fh_SL = np.zeros(N)
    
    # Apply filter size factor to control filter extent
    effective_N = int(N * filter_size_factor)
    center = N // 2
    start_idx = max(0, center - effective_N // 2)
    end_idx = min(N, center + effective_N // 2)
    
    for k1 in range(start_idx, end_idx):
        denominator = 4*(k1-N/2.0)**2-1
        if abs(denominator) > 1e-10:  # Avoid division by zero
            fh_SL[k1] = -2.0/(PI*PI*d*d*denominator)
        else:
            fh_SL[k1] = 0.0
    
    return fh_SL

def filter_SL_proper(N, d, filter_size_factor=1.0):
    """Proper Shepp-Logan filter implementation with configurable size"""
    fh_SL = np.zeros(N)
    
    # Apply filter size factor
    effective_N = int(N * filter_size_factor)
    center = N // 2
    start_idx = max(0, center - effective_N // 2)
    end_idx = min(N, center + effective_N // 2)
    
    for k1 in range(start_idx, end_idx):
        if k1 == N//2:
            # DC component
            fh_SL[k1] = 1.0 / (4.0 * d * d)
        elif (k1 - N//2) % 2 != 0:
            # Odd frequencies
            fh_SL[k1] = -1.0 / (PI * PI * d * d * (k1 - N//2) ** 2)
    
    return fh_SL

def filter_SL_configurable(N, d, filter_type='shepp_logan', filter_size_factor=1.0, cutoff_freq=None):
    """Configurable filter with multiple types and size control"""
    fh_SL = np.zeros(N)
    
    # Determine effective filter size
    if cutoff_freq is not None:
        # Use cutoff frequency to determine filter size
        effective_N = min(N, int(2 * cutoff_freq * N / (1.0 / d)))
    else:
        # Use filter size factor
        effective_N = int(N * filter_size_factor)
    
    center = N // 2
    start_idx = max(0, center - effective_N // 2)
    end_idx = min(N, center + effective_N // 2)
    
    if filter_type == 'shepp_logan':
        for k1 in range(start_idx, end_idx):
            denominator = 4*(k1-N/2.0)**2-1
            if abs(denominator) > 1e-10:
                fh_SL[k1] = -2.0/(PI*PI*d*d*denominator)
    
    elif filter_type == 'ram_lak':
        # Ramp filter (Ram-Lak)
        for k1 in range(start_idx, end_idx):
            if k1 == center:
                fh_SL[k1] = 1.0 / (4.0 * d * d)
            else:
                fh_SL[k1] = -1.0 / (PI * PI * d * d * (k1 - center) ** 2)
    
    elif filter_type == 'cosine':
        # Cosine filter
        for k1 in range(start_idx, end_idx):
            if k1 == center:
                fh_SL[k1] = 1.0 / (4.0 * d * d)
            else:
                freq = (k1 - center) / (N * d)
                fh_SL[k1] = -1.0 / (PI * PI * d * d * (k1 - center) ** 2) * np.cos(PI * freq * d)
    
    elif filter_type == 'hamming':
        # Hamming windowed filter
        for k1 in range(start_idx, end_idx):
            if k1 == center:
                fh_SL[k1] = 1.0 / (4.0 * d * d)
            else:
                # Basic filter
                base_filter = -1.0 / (PI * PI * d * d * (k1 - center) ** 2)
                # Apply Hamming window
                window_val = 0.54 + 0.46 * np.cos(2 * PI * (k1 - center) / effective_N)
                fh_SL[k1] = base_filter * window_val
    
    return fh_SL

def nearestPowerOf2(N):
    """Original power of 2 function"""
    a = int(math.log2(N))
    if 2**a == N:
        return N
    return 2**(a + 1)

def Fun_Weigth_Projection(projection_beta, SOD, delta_dd):
    """Original weighting function"""
    Nrows, Ncolumns = projection_beta.shape
    dd_column = delta_dd*np.arange(-Ncolumns/2+0.5, (Ncolumns/2+1)-0.5, 1.0)
    dd_row = delta_dd*np.arange(-Nrows/2+0.5, (Nrows/2+1)-0.5, 1.0)
    dd_row2D, dd_column2D = np.meshgrid(dd_row, dd_column, indexing='ij')
    weighted_projection = projection_beta*SOD/np.sqrt(SOD*SOD+np.power(dd_row2D, 2.0)+np.power(dd_column2D, 2.0))
    return weighted_projection

def optimize_convolution_configurable(weighted_projection, fh_RL, filter_size_override=None):
    """Enhanced convolution with optional filter size override"""
    Nrows, Ncolumns = weighted_projection.shape
    
    if filter_size_override is not None:
        Nfft = filter_size_override
    else:
        Nfft = nearestPowerOf2(2 * Ncolumns - 1)
    
    # Ensure filter is the right size
    if len(fh_RL) != Nfft:
        fh_RL_padded = np.zeros(Nfft)
        fh_RL_padded[:min(len(fh_RL), Nfft)] = fh_RL[:min(len(fh_RL), Nfft)]
        fh_RL = fh_RL_padded
    
    fh_RL_padded = fh_RL / 2.0
    fh_RL_fft = fft(fh_RL_padded)
    
    projection_padded = np.zeros((Nrows, Nfft))
    projection_padded[:, :Ncolumns] = weighted_projection

    projection_fft = fft(projection_padded, axis=1)
    convoluted_freq = projection_fft * fh_RL_fft
    convoluted_time = ifft(convoluted_freq, axis=1).real
    filtered_projection = convoluted_time[:, :Ncolumns]
    
    return filtered_projection

# Test different filter configurations
print("TESTING CONFIGURABLE FILTER IMPLEMENTATIONS:")
test_N = 512
test_d = 0.1

print(f"\n📊 Filter Size Factor Tests (N={test_N}, d={test_d}):")
for factor in [0.25, 0.5, 0.75, 1.0]:
    test_filter = filter_SL(test_N, test_d, filter_size_factor=factor)
    non_zero = np.count_nonzero(test_filter)
    print(f"  Factor {factor:.2f}: {non_zero} non-zero elements, range: {test_filter.min():.6f} to {test_filter.max():.6f}")

print(f"\n🎛️ Filter Type Tests:")
for ftype in ['shepp_logan', 'ram_lak', 'cosine', 'hamming']:
    test_filter = filter_SL_configurable(test_N, test_d, filter_type=ftype, filter_size_factor=0.5)
    non_zero = np.count_nonzero(test_filter)
    print(f"  {ftype}: {non_zero} non-zero elements, range: {test_filter.min():.6f} to {test_filter.max():.6f}")

In [ ]:
# Global mask storage
GLOBAL_MASK = None
GLOBAL_MASK_INDICES = None

def compute_global_mask(SOD, delta_dd, Nimage, Nrows, Ncolumns):
    """Compute mask once for all projections"""
    MX, MZ = Nimage, int(Nimage*Nrows/Ncolumns)
    
    roi = delta_dd*np.array([-Ncolumns/2.0+0.5, Ncolumns/2.0-0.5, -Nrows/2.0+0.5, Nrows/2.0-0.5])
    hx = (roi[1]-roi[0])/(MX-1)
    xrange = roi[0]+hx*np.arange(0, MX)
    hy = (roi[3]-roi[2])/(MZ-1)
    yrange = roi[2]+hy*np.arange(0, MZ)
    XX, YY, ZZ = np.meshgrid(xrange, xrange, yrange, indexing='ij')
    
    # For any angle, we check the bounds based on the maximum possible extent
    # We use a conservative mask that works for all angles
    max_extent_x = np.max(np.abs(xrange))
    max_extent_y = np.max(np.abs(xrange))
    max_extent_z = np.max(np.abs(yrange))
    
    # Conservative bounds checking
    U_min = (SOD - max_extent_x - max_extent_y)/SOD  # Minimum U value
    a_max = (max_extent_x + max_extent_y)/U_min  # Maximum a value
    b_max = max_extent_z/U_min  # Maximum b value
    
    xx_max = int(np.floor(a_max/delta_dd)) + int(Ncolumns/2)
    yy_max = int(np.floor(b_max/delta_dd)) + int(Nrows/2)
    
    # Create a conservative mask that ensures indices are always valid
    mask_indices = []
    for i in range(MX):
        for j in range(MX):
            for k in range(MZ):
                # Conservative check - if any reasonable projection angle could cause out-of-bounds
                if (xx_max < Ncolumns-1 and yy_max < Nrows-1):
                    mask_indices.append((i, j, k))
    
    return mask_indices

def Fun_BackProjection_with_global_mask(filtered_projection, SOD, beta_num, beta_m, delta_dd, Nimage, use_global_mask=True):
    """Original backprojection function with optional global mask"""
    global GLOBAL_MASK, GLOBAL_MASK_INDICES
    
    Nrows, Ncolumns = filtered_projection.shape
    MX, MZ = Nimage, int(Nimage*Nrows/Ncolumns)
    
    roi = delta_dd*np.array([-Ncolumns/2.0+0.5, Ncolumns/2.0-0.5, -Nrows/2.0+0.5, Nrows/2.0-0.5])
    hx = (roi[1]-roi[0])/(MX-1)
    xrange = roi[0]+hx*np.arange(0, MX)
    hy = (roi[3]-roi[2])/(MZ-1)
    yrange = roi[2]+hy*np.arange(0, MZ)
    XX, YY, ZZ = np.meshgrid(xrange, xrange, yrange, indexing='ij')
    temp_rec = np.zeros((MX, MX, MZ))
    
    U = (SOD+XX*np.sin(beta_m)-YY*np.cos(beta_m))/SOD
    a = (XX*np.cos(beta_m)+YY*np.sin(beta_m))/U
    xx = np.int32(np.floor(a/delta_dd))
    u1 = a/delta_dd-xx
    b = ZZ/U
    yy = np.int32(np.floor(b/delta_dd))
    u2 = b/delta_dd-yy
    xx = xx+int(Ncolumns/2)
    yy = yy+int(Nrows/2)

    if use_global_mask and GLOBAL_MASK is not None:
        # Use pre-computed global mask
        mask = GLOBAL_MASK
    else:
        # Compute mask for this projection (original behavior)
        mask = np.where((xx >= 0) & (xx < Ncolumns-1) & (yy >= 0) & (yy < Nrows-1))
        if not use_global_mask:
            print(f"Mask shape: {len(mask[0])} valid voxels")
    
    xx_masked = xx[mask]
    yy_masked = yy[mask]
    u1_masked = u1[mask]
    u2_masked = u2[mask]
    
    # Ensure indices are within bounds
    valid_indices = (xx_masked >= 0) & (xx_masked < Ncolumns-1) & (yy_masked >= 0) & (yy_masked < Nrows-1)
    xx_masked = xx_masked[valid_indices]
    yy_masked = yy_masked[valid_indices]
    u1_masked = u1_masked[valid_indices]
    u2_masked = u2_masked[valid_indices]
    
    if len(xx_masked) > 0:
        temp = ((1-u1_masked)*(1-u2_masked)*filtered_projection[yy_masked, xx_masked]+
                (1-u1_masked)*u2_masked*filtered_projection[yy_masked+1, xx_masked]+
                (1-u2_masked)*u1_masked*filtered_projection[yy_masked, xx_masked+1]+
                u1_masked*u2_masked*filtered_projection[yy_masked+1, xx_masked+1])
        
        # Create a sub-mask for the valid indices
        final_mask = tuple(arr[valid_indices] for arr in mask)
        temp_rec[final_mask] = temp_rec[final_mask] + temp/(np.power(U[final_mask], 2))*2*PI/beta_num
    
    return temp_rec

# Keep original function for comparison
def Fun_BackProjection(filtered_projection, SOD, beta_num, beta_m, delta_dd, Nimage):
    """Original backprojection function - """
    return Fun_BackProjection_with_global_mask(filtered_projection, SOD, beta_num, beta_m, delta_dd, Nimage, use_global_mask=False)

## Data Loading and Processing

In [49]:
def load_dicom_fast(path_list, max_workers=8):
    """Fast DICOM loading with threading"""
    def read_dicom_data(fname):
        try:
            dcm = dcmread(fname)
            return dcm.pixel_array.astype(np.uint16), float(dcm.GantryAngle)
        except Exception as e:
            print(f"Error reading {fname}: {e}")
            return None, None
    
    images = []
    angles = []
    
    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        with tqdm(total=len(path_list), desc="Loading DICOM files") as pbar:
            future_to_path = {executor.submit(read_dicom_data, path): path for path in path_list}
            
            for future in future_to_path:
                img, angle = future.result()
                if img is not None:
                    images.append(img)
                    angles.append(angle)
                pbar.update(1)
    
    return np.array(images), np.array(angles)

In [ ]:
import cv2

def process_differential_unrotated(images, angles, threshold=10000):
    """Revert to unrotated method - exactly as original but without rotation"""
    n_images = len(images)
    shape = images[0].shape
    processed_images = np.zeros((n_images, shape[0], shape[0]), dtype=np.uint16)
    processed_angles = []
    
    prev = np.zeros((shape[0], shape[0]), dtype=np.uint16)
    
    for idx in tqdm(range(n_images), desc="Processing differences (unrotated)"):
        curr = images[idx]
        _m = curr - prev
        
        if np.max(_m) > threshold:
            # Use previous processed image when threshold exceeded (no rotation)
            if idx > 0:
                processed_images[idx, :, :] = processed_images[idx-1, :, :]
                processed_angles.append(processed_angles[idx-1])
            else:
                processed_images[idx, :, :] = curr - prev
                processed_angles.append(angles[idx])
        else:
            # Normal differential processing
            processed_images[idx, :, :] = curr - prev
            processed_angles.append(angles[idx])
        
        # Always update prev
        prev = curr
    
    return processed_images, np.array(processed_angles)

# Let's examine a DICOM file to understand the metadata
def examine_dicom_metadata(dicom_path):
    """Examine DICOM metadata thoroughly"""
    print("="*60)
    print(f"EXAMINING DICOM METADATA: {dicom_path}")
    print("="*60)
    
    dcm = dcmread(dicom_path)
    
    # Core imaging parameters
    print("\n📊 CORE IMAGING PARAMETERS:")
    print(f"  Modality: {getattr(dcm, 'Modality', 'N/A')}")
    print(f"  RT Image Label: {getattr(dcm, 'RTImageLabel', 'N/A')}")
    print(f"  RT Image Plane: {getattr(dcm, 'RTImagePlane', 'N/A')}")
    print(f"  RT Image Position: {getattr(dcm, 'RTImagePosition', 'N/A')}")
    print(f"  RT Image Orientation: {getattr(dcm, 'RTImageOrientation', 'N/A')}")
    
    # Geometry parameters
    print("\n📐 GEOMETRY PARAMETERS:")
    print(f"  RT Image SID: {getattr(dcm, 'RTImageSID', 'N/A')}")
    print(f"  Radiation Machine SAD: {getattr(dcm, 'RadiationMachineSAD', 'N/A')}")
    print(f"  Gantry Angle: {getattr(dcm, 'GantryAngle', 'N/A')}")
    print(f"  Patient Support Angle: {getattr(dcm, 'PatientSupportAngle', 'N/A')}")
    print(f"  Table Top Eccentric Angle: {getattr(dcm, 'TableTopEccentricAngle', 'N/A')}")
    
    # Image parameters
    print("\n🖼️ IMAGE PARAMETERS:")
    print(f"  Rows: {getattr(dcm, 'Rows', 'N/A')}")
    print(f"  Columns: {getattr(dcm, 'Columns', 'N/A')}")
    print(f"  Pixel Spacing: {getattr(dcm, 'PixelSpacing', 'N/A')}")
    print(f"  Image Plane Pixel Spacing: {getattr(dcm, 'ImagePlanePixelSpacing', 'N/A')}")
    print(f"  Bits Allocated: {getattr(dcm, 'BitsAllocated', 'N/A')}")
    print(f"  Bits Stored: {getattr(dcm, 'BitsStored', 'N/A')}")
    print(f"  Pixel Representation: {getattr(dcm, 'PixelRepresentation', 'N/A')}")
    
    # Dose/calibration parameters
    print("\n⚡ DOSE/CALIBRATION PARAMETERS:")
    print(f"  Primary Dosimeter Unit: {getattr(dcm, 'PrimaryDosimeterUnit', 'N/A')}")
    print(f"  RT Image Description: {getattr(dcm, 'RTImageDescription', 'N/A')}")
    print(f"  Exposure: {getattr(dcm, 'Exposure', 'N/A')}")
    print(f"  X-Ray Tube Current: {getattr(dcm, 'XRayTubeCurrent', 'N/A')}")
    print(f"  KVP: {getattr(dcm, 'KVP', 'N/A')}")
    
    # Image position/orientation in 3D space
    print("\n🌐 3D POSITIONING:")
    print(f"  Image Position Patient: {getattr(dcm, 'ImagePositionPatient', 'N/A')}")
    print(f"  Image Orientation Patient: {getattr(dcm, 'ImageOrientationPatient', 'N/A')}")
    
    # Acquisition parameters
    print("\n🎯 ACQUISITION PARAMETERS:")
    print(f"  Acquisition Date: {getattr(dcm, 'AcquisitionDate', 'N/A')}")
    print(f"  Acquisition Time: {getattr(dcm, 'AcquisitionTime', 'N/A')}")
    print(f"  Instance Number: {getattr(dcm, 'InstanceNumber', 'N/A')}")
    print(f"  Series Number: {getattr(dcm, 'SeriesNumber', 'N/A')}")
    
    # Check pixel data
    print("\n💾 PIXEL DATA:")
    pixel_array = dcm.pixel_array
    print(f"  Pixel array shape: {pixel_array.shape}")
    print(f"  Pixel array dtype: {pixel_array.dtype}")
    print(f"  Value range: {pixel_array.min()} to {pixel_array.max()}")
    print(f"  Mean value: {pixel_array.mean():.2f}")
    
    return dcm

## Reconstruction Classes

In [ ]:
class FilteredBackprojectionReconstructor:
    """True Filtered Backprojection reconstructor with configurable filters"""
    
    def __init__(self, SOD, delta_dd, Nimage):
        self.SOD = SOD
        self.delta_dd = delta_dd
        self.Nimage = Nimage
        self.global_mask_initialized = False
    
    def initialize_global_mask(self, projections):
        """Initialize the global mask once for all projections"""
        global GLOBAL_MASK
        
        Nrows, Ncolumns = projections.shape[1], projections.shape[2]
        MX, MZ = self.Nimage, int(self.Nimage * Nrows / Ncolumns)
        
        # Create coordinate grids once
        roi = self.delta_dd*np.array([-Ncolumns/2.0+0.5, Ncolumns/2.0-0.5, -Nrows/2.0+0.5, Nrows/2.0-0.5])
        hx = (roi[1]-roi[0])/(MX-1)
        xrange = roi[0]+hx*np.arange(0, MX)
        hy = (roi[3]-roi[2])/(MZ-1)
        yrange = roi[2]+hy*np.arange(0, MZ)
        XX, YY, ZZ = np.meshgrid(xrange, xrange, yrange, indexing='ij')
        
        # Compute a conservative mask that works for most projections
        U = (self.SOD + XX*np.sin(0) - YY*np.cos(0))/self.SOD
        a = (XX*np.cos(0) + YY*np.sin(0))/U
        xx = np.int32(np.floor(a/self.delta_dd))
        b = ZZ/U
        yy = np.int32(np.floor(b/self.delta_dd))
        xx = xx + int(Ncolumns/2)
        yy = yy + int(Nrows/2)
        
        # Create a slightly more conservative mask
        margin = 5  # pixels margin for safety
        GLOBAL_MASK = np.where((xx >= margin) & (xx < Ncolumns-1-margin) & 
                              (yy >= margin) & (yy < Nrows-1-margin))
        
        self.global_mask_initialized = True
        print(f"Global mask initialized with {len(GLOBAL_MASK[0])} valid voxels")
        return GLOBAL_MASK
    
    def reconstruct_filtered_backprojection(self, projections, angles, 
                                          filter_type='shepp_logan',
                                          filter_size_factor=1.0,
                                          use_weighting=True,
                                          chunk_size=25, 
                                          use_global_mask=True):
        """
        True Filtered Backprojection reconstruction
        
        Parameters:
        - filter_type: 'shepp_logan', 'ram_lak', 'cosine', 'hamming'
        - filter_size_factor: 0.1 to 1.0 (controls filter extent)
        - use_weighting: Whether to apply geometric weighting before filtering
        """
        n_angles = len(angles)
        beta_rad = angles * PI / 180.0
        
        # Initialize global mask if requested
        if use_global_mask and not self.global_mask_initialized:
            self.initialize_global_mask(projections)
        
        # Prepare configurable filter
        Ncolumns = projections.shape[2]
        Nrows = projections.shape[1]
        Nfft = nearestPowerOf2(2 * Ncolumns - 1)
        
        fh_RL = filter_SL_configurable(
            Nfft, 
            self.delta_dd, 
            filter_type=filter_type, 
            filter_size_factor=filter_size_factor
        )
        
        print(f"FILTERED BACKPROJECTION RECONSTRUCTION:")
        print(f"  Projections: {n_angles}")
        print(f"  Filter type: {filter_type}")
        print(f"  Filter size factor: {filter_size_factor}")
        print(f"  Filter size: {len(fh_RL)}, Nfft: {Nfft}")
        print(f"  Filter non-zero elements: {np.count_nonzero(fh_RL)}")
        print(f"  Filter range: {fh_RL.min():.6f} to {fh_RL.max():.6f}")
        print(f"  Use weighting: {use_weighting}")
        print(f"  Use global mask: {use_global_mask}")
        
        # Initialize result
        MX = self.Nimage
        MZ = int(self.Nimage * Nrows / Ncolumns)
        rec_image = np.zeros((MX, MX, MZ))
        
        # Process in chunks
        chunks = [list(range(i, min(i + chunk_size, n_angles))) 
                 for i in range(0, n_angles, chunk_size)]
        
        print(f"  Processing in {len(chunks)} chunks...")
        
        for chunk_idx, chunk_indices in enumerate(tqdm(chunks, desc="Filtered backprojection")):
            chunk_result = np.zeros((MX, MX, MZ))
            
            for i, angle_idx in enumerate(chunk_indices):
                projection_beta = projections[angle_idx, :, :]
                
                # Step 1: Optional geometric weighting
                if use_weighting:
                    weighted_projection = Fun_Weigth_Projection(projection_beta, self.SOD, self.delta_dd)
                    input_projection = weighted_projection
                else:
                    input_projection = projection_beta
                
                # Step 2: Filter the projection
                filtered_projection = optimize_convolution_configurable(input_projection, fh_RL)
                
                # Step 3: TRUE FILTERED BACKPROJECTION - backproject the filtered data
                if use_global_mask:
                    temp_rec = Fun_BackProjection_with_global_mask(
                        filtered_projection,  # ✅ NOW USING FILTERED PROJECTION!
                        self.SOD, 
                        n_angles,
                        beta_rad[angle_idx],
                        self.delta_dd, 
                        self.Nimage,
                        use_global_mask=True
                    )
                else:
                    temp_rec = Fun_BackProjection(
                        filtered_projection,  # ✅ NOW USING FILTERED PROJECTION!
                        self.SOD, 
                        n_angles,
                        beta_rad[angle_idx],
                        self.delta_dd, 
                        self.Nimage
                    )
                
                chunk_result += temp_rec
            
            rec_image += chunk_result
        
        return rec_image

# For comparison - the old FDK approach
class FDKReconstructor(FilteredBackprojectionReconstructor):
    """Original FDK approach for comparison"""
    
    def reconstruct_fdk(self, projections, angles, 
                       filter_type='shepp_logan',
                       filter_size_factor=1.0,
                       chunk_size=25, 
                       use_global_mask=True):
        """Original FDK reconstruction (backprojects weighted, not filtered)"""
        n_angles = len(angles)
        beta_rad = angles * PI / 180.0
        
        if use_global_mask and not self.global_mask_initialized:
            self.initialize_global_mask(projections)
        
        Ncolumns = projections.shape[2]
        Nrows = projections.shape[1]
        Nfft = nearestPowerOf2(2 * Ncolumns - 1)
        
        fh_RL = filter_SL_configurable(
            Nfft, self.delta_dd, 
            filter_type=filter_type, 
            filter_size_factor=filter_size_factor
        )
        
        print(f"FDK RECONSTRUCTION (for comparison):")
        print(f"  Filter type: {filter_type}, size factor: {filter_size_factor}")
        
        MX = self.Nimage
        MZ = int(self.Nimage * Nrows / Ncolumns)
        rec_image = np.zeros((MX, MX, MZ))
        
        chunks = [list(range(i, min(i + chunk_size, n_angles))) 
                 for i in range(0, n_angles, chunk_size)]
        
        for chunk_idx, chunk_indices in enumerate(tqdm(chunks, desc="FDK reconstruction")):
            chunk_result = np.zeros((MX, MX, MZ))
            
            for angle_idx in chunk_indices:
                projection_beta = projections[angle_idx, :, :]
                
                # Weight and filter
                weighted_projection = Fun_Weigth_Projection(projection_beta, self.SOD, self.delta_dd)
                filtered_projection = optimize_convolution_configurable(weighted_projection, fh_RL)
                
                # FDK: backproject the weighted (not filtered) projection
                if use_global_mask:
                    temp_rec = Fun_BackProjection_with_global_mask(
                        weighted_projection,  # ❗ FDK uses weighted, not filtered
                        self.SOD, n_angles, beta_rad[angle_idx], 
                        self.delta_dd, self.Nimage, use_global_mask=True
                    )
                else:
                    temp_rec = Fun_BackProjection(
                        weighted_projection,  # ❗ FDK uses weighted, not filtered
                        self.SOD, n_angles, beta_rad[angle_idx], 
                        self.delta_dd, self.Nimage
                    )
                
                chunk_result += temp_rec
            
            rec_image += chunk_result
        
        return rec_image

print("✅ Filtered Backprojection Reconstructor ready!")
print("   - True filtered backprojection: backprojects the filtered projection")
print("   - Configurable filters: shepp_logan, ram_lak, cosine, hamming") 
print("   - Adjustable filter size: factor 0.1 to 1.0")
print("   - Optional geometric weighting")
print("   - Global mask for consistency")

## Data Loading

In [52]:
# Data path - single folder for SIB COMPLEX TARGET
_data_pth = r"E:\CMC\pyprojects\radio_therapy\dose-3d\dataset\VMAT 2025 - 6. SIB COMPLEX TARGET\T1\873251691"

# Get all DICOM files from the single folder
_files = [f for f in os.listdir(_data_pth) if f.endswith('.dcm')]
_pth = [os.path.join(_data_pth, f) for f in _files]

print(f"Processing {len(_pth)} DICOM files from SIB COMPLEX TARGET dataset")

# Load and process data with original algorithm
start_time = time.time()
raw_images, raw_angles = load_dicom_fast(_pth, max_workers=6)
print(f"DICOM loading: {time.time() - start_time:.2f}s")

start_time = time.time()
processed_images, processed_angles = process_differential_original(raw_images, raw_angles)
print(f"Differential processing: {time.time() - start_time:.2f}s")

# # Sort by angle - EXACTLY as original
# sorted_indices = np.argsort(processed_angles)
# sorted_images = np.zeros((len(_pth), processed_images.shape[1], processed_images.shape[1]), dtype=np.uint16)

# for idx, val in enumerate(tqdm(sorted_indices, desc="Sorting by angle")):
#     sorted_images[idx, :, :] = processed_images[val, :, :]

# sorted_angles = processed_angles[sorted_indices]

# print(f"Final data shape: {sorted_images.shape}")
# print(f"Angle range: {sorted_angles.min():.1f}° to {sorted_angles.max():.1f}°")
# print(f"Data type: {sorted_images.dtype}")

# Clean up memory
# del raw_images, processed_images

sorted_images = processed_images
sorted_angles = processed_angles

Processing 415 DICOM files from SIB COMPLEX TARGET dataset


Loading DICOM files: 100%|██████████| 415/415 [00:00<00:00, 435.83it/s]


DICOM loading: 1.74s


Processing differences:  22%|██▏       | 93/415 [00:00<00:01, 208.90it/s]

Processing image 73/415 with angle 239.71469642569 degrees


...

Processing image 152/415 with angle 308.736590321357 degrees


Processing differences:  89%|████████▉ | 369/415 [00:01<00:00, 287.44it/s]

Processing image 325/415 with angle 105.146962089456 degrees


Processing differences: 100%|██████████| 415/415 [00:01<00:00, 248.82it/s]

Differential processing: 1.67s


In [ ]:
# EXAMINE DICOM METADATA AND REVERT TO UNROTATED PROCESSING
print("="*70)
print("REVERTING TO UNROTATED METHOD AND EXAMINING DICOM METADATA")
print("="*70)

# First, examine a DICOM file to understand what we might be missing
sample_dicom_path = _pth[0]  # First DICOM file
sample_dcm = examine_dicom_metadata(sample_dicom_path)

print("\n" + "="*50)
print("REPROCESSING WITH UNROTATED METHOD")
print("="*50)

# Reprocess with unrotated method
unrotated_images, unrotated_angles = process_differential_unrotated(
    raw_images, raw_angles, 
    threshold=10000
)

# Sort by angle
sorted_indices_unrot = np.argsort(unrotated_angles)
sorted_images_unrot = unrotated_images[sorted_indices_unrot]
sorted_angles_unrot = unrotated_angles[sorted_indices_unrot]

print(f"\nUnrotated processing results:")
print(f"  Total images: {len(sorted_images_unrot)}")
print(f"  Angle range: {sorted_angles_unrot.min():.1f}° to {sorted_angles_unrot.max():.1f}°")

# Analyze for issues
duplicates = analyze_processed_data(sorted_images_unrot, sorted_angles_unrot)

# Compare reconstruction parameters with what the GUI uses
print(f"\n" + "="*50)
print("COMPARING RECONSTRUCTION PARAMETERS")
print("="*50)

# From GUI code
gui_width = 0.172  # mm (hardcoded in GUI)
gui_delta_dd = gui_width * sample_dcm.RadiationMachineSAD / sample_dcm.RTImageSID

# From DICOM
dicom_pixel_spacing = getattr(sample_dcm, 'ImagePlanePixelSpacing', None)
if dicom_pixel_spacing is None:
    dicom_pixel_spacing = getattr(sample_dcm, 'PixelSpacing', None)

print(f"GUI approach:")
print(f"  Width: {gui_width} mm (hardcoded)")
print(f"  SAD: {sample_dcm.RadiationMachineSAD} mm")
print(f"  SID: {sample_dcm.RTImageSID} mm") 
print(f"  Delta_dd: {gui_delta_dd:.6f} mm")

print(f"\nDICOM metadata approach:")
print(f"  Pixel spacing from DICOM: {dicom_pixel_spacing}")
if dicom_pixel_spacing:
    dicom_delta_dd = float(dicom_pixel_spacing[0]) * sample_dcm.RadiationMachineSAD / sample_dcm.RTImageSID
    print(f"  Calculated delta_dd: {dicom_delta_dd:.6f} mm")
    print(f"  Difference: {abs(gui_delta_dd - dicom_delta_dd):.6f} mm")

# Set the final data for reconstruction
sorted_images = sorted_images_unrot
sorted_angles = sorted_angles_unrot

print(f"\n✅ Reverted to unrotated processing method")
print(f"📊 Data ready for reconstruction: {len(sorted_images)} images")

# STOP_HERE

## Filter Visualization

# Create visualization of different filter types and sizes
fig, axes = plt.subplots(2, 4, figsize=(16, 8))
fig.suptitle('Reconstruction Filter Comparison', fontsize=16)

# Test parameters
test_N = 512
test_d = 0.1
x_axis = np.arange(test_N) - test_N//2

# Filter types and sizes to test
filter_configs = [
    ('shepp_logan', 1.0), ('shepp_logan', 0.5), ('shepp_logan', 0.25),
    ('ram_lak', 0.5), ('cosine', 0.5), ('hamming', 0.5),
    ('shepp_logan', 0.1), ('ram_lak', 0.25)
]

for i, (filter_type, size_factor) in enumerate(filter_configs):
    row = i // 4
    col = i % 4
    
    if row < 2 and col < 4:  # Only plot if within subplot grid
        # Generate filter
        test_filter = filter_SL_configurable(test_N, test_d, 
                                           filter_type=filter_type, 
                                           filter_size_factor=size_factor)
        
        # Plot filter
        axes[row, col].plot(x_axis, test_filter, 'b-', linewidth=1.5)
        axes[row, col].set_title(f'{filter_type}\nSize: {size_factor}', fontsize=10)
        axes[row, col].grid(True, alpha=0.3)
        axes[row, col].set_xlim(-test_N//4, test_N//4)
        
        # Set y-axis limits for better visualization
        if np.any(test_filter != 0):
            y_max = np.max(np.abs(test_filter[test_filter != 0]))
            axes[row, col].set_ylim(-y_max*1.1, y_max*1.1)
        
        # Add statistics text
        non_zero = np.count_nonzero(test_filter)
        axes[row, col].text(0.02, 0.98, f'Non-zero: {non_zero}', 
                          transform=axes[row, col].transAxes, 
                          verticalalignment='top', fontsize=8,
                          bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.8))

# Remove empty subplots
for i in range(len(filter_configs), 8):
    row = i // 4
    col = i % 4
    if row < 2 and col < 4:
        fig.delaxes(axes[row, col])

plt.tight_layout()
plt.show()

# Summary of filter characteristics
print("FILTER CHARACTERISTICS SUMMARY:")
print("="*50)
print("Filter Type    | Size Factor | Non-zero Elements | Max Value")
print("-"*50)

for filter_type, size_factor in filter_configs:
    test_filter = filter_SL_configurable(test_N, test_d, 
                                       filter_type=filter_type, 
                                       filter_size_factor=size_factor)
    non_zero = np.count_nonzero(test_filter)
    max_val = np.max(np.abs(test_filter)) if np.any(test_filter != 0) else 0
    
    print(f"{filter_type:<13} | {size_factor:<11} | {non_zero:<17} | {max_val:.6f}")

print("\nFILTER RECOMMENDATIONS:")
print("• Shepp-Logan: Good general purpose filter")
print("• Ram-Lak: Sharp, high-frequency preservation")  
print("• Cosine: Smoother than Ram-Lak, less noise")
print("• Hamming: Smoothest, best noise reduction")
print("• Smaller size factors: Less noise, smoother images")
print("• Larger size factors: More detail, potentially more noise")

In [ ]:
# FILTERED BACKPROJECTION WITH CONFIGURABLE FILTER SIZES
print("="*70)
print("TRUE FILTERED BACKPROJECTION WITH CONFIGURABLE FILTERS")
print("="*70)

# Get reconstruction parameters
dcm = dcmread(_pth[0])
SID = dcm.RTImageSID
SAD = dcm.RadiationMachineSAD
SOD = SAD
SDD = SID
width = 0.172  # mm
delta_dd = width * SOD / SDD
Nimage = 100

print(f"Reconstruction parameters:")
print(f"  Image size: {Nimage}x{Nimage}")
print(f"  SOD: {SOD} mm, SDD: {SDD} mm, Delta_dd: {delta_dd:.6f} mm")
print(f"  Using unrotated processed data: {len(sorted_angles)} projections")

# Create filtered backprojection reconstructor
fbp_reconstructor = FilteredBackprojectionReconstructor(SOD, delta_dd, Nimage)

# Test different filter configurations
test_configs = [
    {'filter_type': 'shepp_logan', 'filter_size_factor': 0.5, 'use_weighting': True},
    {'filter_type': 'shepp_logan', 'filter_size_factor': 0.25, 'use_weighting': True},
    {'filter_type': 'ram_lak', 'filter_size_factor': 0.5, 'use_weighting': True},
    {'filter_type': 'hamming', 'filter_size_factor': 0.5, 'use_weighting': True},
]

results = {}

for i, config in enumerate(test_configs):
    print(f"\n🔧 TEST {i+1}: {config}")
    print(f"="*50)
    
    start_time = time.time()
    rec_image = fbp_reconstructor.reconstruct_filtered_backprojection(
        sorted_images, sorted_angles,
        filter_type=config['filter_type'],
        filter_size_factor=config['filter_size_factor'],
        use_weighting=config['use_weighting'],
        chunk_size=25,
        use_global_mask=True
    )
    
    total_time = time.time() - start_time
    
    # Calculate quality metrics
    non_zero_voxels = np.count_nonzero(rec_image)
    total_voxels = rec_image.size
    coverage = non_zero_voxels / total_voxels * 100
    
    config_name = f"{config['filter_type']}_f{config['filter_size_factor']}"
    results[config_name] = {
        'image': rec_image,
        'time': total_time,
        'coverage': coverage,
        'min_val': rec_image.min(),
        'max_val': rec_image.max(),
        'mean_nonzero': rec_image[rec_image > 0].mean() if np.any(rec_image > 0) else 0,
        'config': config
    }
    
    print(f"\nResults for {config_name}:")
    print(f"  Time: {total_time:.2f} seconds")
    print(f"  Shape: {rec_image.shape}")
    print(f"  Value range: {rec_image.min():.6f} to {rec_image.max():.6f}")
    print(f"  Coverage: {coverage:.1f}% ({non_zero_voxels}/{total_voxels})")
    print(f"  Mean (non-zero): {rec_image[rec_image > 0].mean():.6f}" if np.any(rec_image > 0) else "  No non-zero values")
    print(f"  Center voxel: {rec_image[50, 50, 50]:.6f}")

# Summary comparison
print(f"\n" + "="*70)
print("SUMMARY COMPARISON OF FILTERED BACKPROJECTION CONFIGURATIONS")
print("="*70)

print(f"{'Config':<20} {'Time(s)':<8} {'Coverage(%)':<12} {'Max Value':<12} {'Mean(>0)':<12}")
print(f"{'-'*20} {'-'*8} {'-'*12} {'-'*12} {'-'*12}")

for name, result in results.items():
    print(f"{name:<20} {result['time']:<8.1f} {result['coverage']:<12.1f} {result['max_val']:<12.6f} {result['mean_nonzero']:<12.6f}")

# Store the best result for visualization
best_config = max(results.keys(), key=lambda k: results[k]['coverage'])
rec_image = results[best_config]['image']

print(f"\n🏆 Best configuration: {best_config}")
print(f"   Filter: {results[best_config]['config']['filter_type']}")
print(f"   Size factor: {results[best_config]['config']['filter_size_factor']}")
print(f"   Coverage: {results[best_config]['coverage']:.1f}%")

print(f"\n💡 FILTER SIZE RECOMMENDATIONS:")
print(f"   • Smaller filter (0.25): Less noise, smoother, may lose details")
print(f"   • Medium filter (0.5): Good balance of detail and noise")
print(f"   • Larger filter (1.0): More detail, potentially more noise")
print(f"   • Try different filter types for varying characteristics")

print(f"\n✅ True Filtered Backprojection reconstruction complete!")
print(f"   You can now adjust filter_size_factor from 0.1 to 1.0 for different results")

## Display Reconstruction Results

# Display reconstruction - cross sections
NimageZ = Nimage * sorted_images.shape[1] // sorted_images.shape[2]
Z_c = int(NimageZ // 2)
X_c = int(Nimage // 2)
Y_c = int(Nimage // 2)

fig, axes = plt.subplots(1, 3, figsize=(15, 5))
fig.suptitle('Filtered Backprojection Reconstruction Results', fontsize=14)

# Cross sections
axes[0].imshow(rec_image[30, :, :].T, cmap='hot', origin='lower')
axes[0].set_title('YZ cross section (X=30)')
axes[0].axis('off')

axes[1].imshow(rec_image[:, Y_c, :].T, cmap='hot', origin='lower')
axes[1].set_title(f'XZ cross section (Y={Y_c})')
axes[1].axis('off')

axes[2].imshow(rec_image[:, :, Z_c].T, cmap='hot', origin='lower')
axes[2].set_title(f'XY cross section (Z={Z_c})')
axes[2].axis('off')

plt.tight_layout()
plt.show()

print(f"\nFinal Reconstruction Statistics:")
print(f"Min: {rec_image.min():.6f}")
print(f"Max: {rec_image.max():.6f}")
print(f"Mean: {rec_image.mean():.6f}")
print(f"Std: {rec_image.std():.6f}")
print(f"Non-zero voxels: {np.count_nonzero(rec_image)} / {rec_image.size}")
print(f"Coverage: {np.count_nonzero(rec_image)/rec_image.size*100:.1f}%")

In [ ]:
# Save results
_TPS_pth = r"E:\CMC\pyprojects\radio_therapy\dose-3d\dataset\3DDose\EPID_12_t0.dcm"

try:
    if os.path.exists(_TPS_pth):
        tps_dcm = dcmread(_TPS_pth)
        
        # Scale and prepare for DICOM export
        scaled_image = np.int32(rec_image * 1)
        
        # Create new DICOM based on TPS template
        write_dicom = tps_dcm.copy()
        write_dicom.NumberOfFrames = str(rec_image.shape[2])
        write_dicom.Rows = rec_image.shape[0]
        write_dicom.Columns = rec_image.shape[1]
        write_dicom.PixelData = scaled_image.tobytes()
        
        # Save with timestamp
        import datetime
        timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
        output_path = f"E:\\CMC\\pyprojects\\radio_therapy\\dose-3d\\results\\filtered_backprojection_{timestamp}.dcm"
        
        # Ensure results directory exists
        os.makedirs(os.path.dirname(output_path), exist_ok=True)
        
        # Uncomment to save:
        # dcmwrite(output_path, write_dicom)
        print(f"Ready to save to: {output_path}")
    else:
        print("TPS template not found - cannot create DICOM output")
        
except Exception as e:
    print(f"Error preparing DICOM output: {e}")

print("\n✅ Filtered Backprojection reconstruction complete!")
print("   You can now experiment with different filter types and sizes:")
print("   - Adjust filter_size_factor (0.1 to 1.0)")
print("   - Try different filter_type ('shepp_logan', 'ram_lak', 'cosine', 'hamming')")
print("   - Toggle use_weighting (True/False)")
print("   - Compare with FDK approach if needed")

In [ ]:
## Quick Reconstruction Function

def quick_reconstruction(filter_type='shepp_logan', filter_size_factor=0.5, use_weighting=True):
    """Quick function to test different filter configurations"""
    
    print(f"Running reconstruction with:")
    print(f"  Filter: {filter_type}")
    print(f"  Size factor: {filter_size_factor}")
    print(f"  Weighting: {use_weighting}")
    
    fbp_reconstructor = FilteredBackprojectionReconstructor(SOD, delta_dd, Nimage)
    
    start_time = time.time()
    result = fbp_reconstructor.reconstruct_filtered_backprojection(
        sorted_images, sorted_angles,
        filter_type=filter_type,
        filter_size_factor=filter_size_factor,
        use_weighting=use_weighting,
        chunk_size=25,
        use_global_mask=True
    )
    
    reconstruction_time = time.time() - start_time
    
    # Quick stats
    non_zero = np.count_nonzero(result)
    coverage = non_zero / result.size * 100
    
    print(f"\nResults:")
    print(f"  Time: {reconstruction_time:.1f}s")
    print(f"  Coverage: {coverage:.1f}%")
    print(f"  Range: {result.min():.6f} to {result.max():.6f}")
    print(f"  Center: {result[50,50,50]:.6f}")
    
    return result

# Example usage:
# rec_small = quick_reconstruction('shepp_logan', 0.25, True)
# rec_large = quick_reconstruction('shepp_logan', 0.75, True)
# rec_hamming = quick_reconstruction('hamming', 0.5, True)

print("Use quick_reconstruction() to test different configurations quickly!")

In [ ]:
# Usage Examples

# Quick test with different configurations
print("Example usage:")
print("="*50)

print("\n1. Quick reconstruction:")
print("result = quick_reconstruction('hamming', 0.3, True)")

print("\n2. Full control:")
print("""fbp = FilteredBackprojectionReconstructor(SOD, delta_dd, Nimage)
result = fbp.reconstruct_filtered_backprojection(
    sorted_images, sorted_angles,
    filter_type='shepp_logan',      # Filter type
    filter_size_factor=0.5,         # Filter size
    use_weighting=True,             # Geometric weighting
    chunk_size=25,                  # Memory management
    use_global_mask=True            # Consistent masking
)""")

print("\n3. Available filter types:")
filter_types = ['shepp_logan', 'ram_lak', 'cosine', 'hamming']
for ft in filter_types:
    print(f"   - {ft}")

print("\n4. Recommended filter size factors:")
size_recommendations = [
    (0.1, "Very smooth, minimal noise"),
    (0.25, "Smooth, good noise reduction"),
    (0.5, "Balanced detail and noise"),
    (0.75, "High detail, some noise"),
    (1.0, "Maximum detail, more noise")
]

for size, desc in size_recommendations:
    print(f"   - {size}: {desc}")

print(f"\n✅ The reconstruction now gives you full control over filtering!")
print(f"   Experiment with different combinations to optimize your results.")

## Summary

This notebook implements **true filtered backprojection** with the following features:

### ✅ Key Improvements:
1. **Fixed Shepp-Logan Filter**: Removed the bug that was zeroing out the filter
2. **True Filtered Backprojection**: Backprojects the filtered projection (not weighted)  
3. **Configurable Filter Sizes**: Control filter extent with `filter_size_factor` (0.1-1.0)
4. **Multiple Filter Types**: Shepp-Logan, Ram-Lak, Cosine, Hamming
5. **Global Mask**: Consistent masking across all projections
6. **Unrotated Processing**: Clean differential processing without rotation artifacts

### 🎛️ Filter Selection Guide:
- **Small filters (0.1-0.3)**: Smoother, less noise, may lose details
- **Medium filters (0.4-0.6)**: Good balance of detail and noise
- **Large filters (0.7-1.0)**: Maximum detail, potentially more noise

### 🔧 Filter Types:
- **Shepp-Logan**: General purpose, good balance
- **Ram-Lak**: Sharp, preserves high frequencies
- **Cosine**: Smoother than Ram-Lak, reduces noise
- **Hamming**: Smoothest, best noise reduction